# Virtual data set (VDS) reference file for ECCO_L4_TEMP_SALINITY_05DEG_DAILY_V4R4 using Virtualizarr

Saves VDS as JSON file. ECCO_L4_TEMP_SALINITY_05DEG_DAILY_V4R4 (https://doi.org/10.5067/ECG5D-OTS44) is an L4 regularly gridded data set.

Note this was run using Zarr v3.

In [1]:
# Built-in packages
import os
import sys
import shutil

# Filesystem management 
import fsspec
import earthaccess

# Data handling
import numpy as np
import xarray as xr
from virtualizarr import open_virtual_dataset
import pandas as pd

# Parallel computing 
import multiprocessing
from dask import delayed
import dask.array as da
from dask.distributed import Client

# Other
import matplotlib.pyplot as plt

## 1. Get Data File S3 endpoints in Earthdata Cloud

In [2]:
# Get Earthdata creds
earthaccess.login()

Enter your Earthdata Login username:  deanh808
Enter your Earthdata password:  ········


In [3]:
# Get AWS creds. Note that if you spend more than 1 hour in the notebook, you may have to re-run this line!!!
fs = earthaccess.get_s3_filesystem(daac="PODAAC")

In [4]:
# Locate CCMP file information / metadata:
granule_info = earthaccess.search_data(
    short_name="ECCO_L4_TEMP_SALINITY_05DEG_DAILY_V4R4",
    )

In [5]:
# Get S3 endpoints for all files:
data_s3links = [g.data_links(access="direct")[0] for g in granule_info]
print("Number of granules found =", len(data_s3links))
print("First few granules:")
data_s3links[0:3]

Number of granules found = 9497
First few granules:


['s3://podaac-ops-cumulus-protected/ECCO_L4_TEMP_SALINITY_05DEG_DAILY_V4R4/OCEAN_TEMPERATURE_SALINITY_day_mean_1992-01-01_ECCO_V4r4_latlon_0p50deg.nc',
 's3://podaac-ops-cumulus-protected/ECCO_L4_TEMP_SALINITY_05DEG_DAILY_V4R4/OCEAN_TEMPERATURE_SALINITY_day_mean_1992-01-02_ECCO_V4r4_latlon_0p50deg.nc',
 's3://podaac-ops-cumulus-protected/ECCO_L4_TEMP_SALINITY_05DEG_DAILY_V4R4/OCEAN_TEMPERATURE_SALINITY_day_mean_1992-01-03_ECCO_V4r4_latlon_0p50deg.nc']

## 2. Generate single-orbit reference files

One file per orbit, so one reference file per orbit.

In [6]:
# This will be assigned to 'loadable_variables' and needs to be modified per the specific 
# coord names of the data set:
coord_vars = ["time", "latitude", "longitude", "Z"]

In [7]:
print("CPU count =", multiprocessing.cpu_count())

CPU count = 16


In [8]:
reader_opts = {"storage_options": fs.storage_options} # S3 filesystem creds from previous section.

In [9]:
# Start up cluster and print some information about it:
client = Client(n_workers=16, threads_per_worker=1)
print(client.cluster)
print("View any work being done on the cluster here", client.dashboard_link)

LocalCluster(ff75e981, 'tcp://127.0.0.1:38279', workers=16, threads=16, memory=60.77 GiB)
View any work being done on the cluster here https://cluster-zzkxl.dask.host/jupyter/proxy/8787/status


In [10]:
%%time
# Create individual references:
open_vds_par = delayed(open_virtual_dataset)
tasks = [
    open_vds_par(p, indexes={}, reader_options=reader_opts, loadable_variables=coord_vars) 
    for p in data_s3links[:]
    ]
virtual_ds_list = list(da.compute(*tasks)) # The xr.combine_nested() function below needs a list rather than a tuple.

CPU times: user 3min 13s, sys: 33.6 s, total: 3min 47s
Wall time: 13min 24s


In [11]:
len(virtual_ds_list)

9497

## 3. Generate combined reference file

In [12]:
%%time
# Create the combined reference
virtual_ds_combined = xr.combine_nested(virtual_ds_list, concat_dim='time', coords='minimal', compat='override', combine_attrs='drop_conflicts')

CPU times: user 7.78 s, sys: 209 ms, total: 7.99 s
Wall time: 7.82 s


In [13]:
# Save in JSON or PARQUET format:
fname_combined_json = 'ECCO_L4_TEMP_SALINITY_05DEG_DAILY_V4R4.json'
virtual_ds_combined.virtualize.to_kerchunk(fname_combined_json, format='json')

## 4. Test combined reference files

In [16]:
def opends_withref(ref, fs_data):
    """
    "ref" is a reference file or object. "fs_data" is a filesystem with credentials to
    access the actual data files. 
    """
    storage_opts = {"fo": ref, "remote_protocol": "s3", "remote_options": fs_data.storage_options}
    fs_ref = fsspec.filesystem('reference', **storage_opts)
    m = fs_ref.get_mapper('')
    data = xr.open_dataset(
        m, engine="zarr", chunks={},
        backend_kwargs={"consolidated": False}
    )
    return data

### 4.1 JSON reference file

In [14]:
%%time
data = xr.open_dataset(
    fname_combined_json,
    engine="kerchunk",
    chunks={},
    backend_kwargs={
        "storage_options": {
            "remote_protocol": "s3",
            "remote_options": fs.storage_options
        }
    }
)
data

/opt/coiled/env/lib/python3.13/site-packages/zarr/core/metadata/v2.py:192: UserWarning: Found an empty list of filters in the array metadata document. This is contrary to the Zarr V2 specification, and will cause an error in the future. Use None (or Null in a JSON document) instead of an empty list of filters.
  warnings.warn(msg, UserWarning, stacklevel=1)


<xarray.Dataset> Size: 985GB
Dimensions:         (time: 9497, Z: 50, latitude: 360, longitude: 720, nv: 2)
Coordinates:
    time_bnds       (time, nv) datetime64[ns] 152kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    latitude_bnds   (latitude, nv) float32 3kB dask.array<chunksize=(360, 2), meta=np.ndarray>
    longitude_bnds  (longitude, nv) float32 6kB dask.array<chunksize=(720, 2), meta=np.ndarray>
    Z_bnds          (Z, nv) float32 400B dask.array<chunksize=(50, 2), meta=np.ndarray>
  * time            (time) datetime64[ns] 76kB 1992-01-01T18:00:00 ... 2017-1...
  * Z               (Z) float32 200B -5.0 -15.0 -25.0 ... -5.461e+03 -5.906e+03
  * latitude        (latitude) float32 1kB -89.75 -89.25 -88.75 ... 89.25 89.75
  * longitude       (longitude) float32 3kB -179.8 -179.2 -178.8 ... 179.2 179.8
Dimensions without coordinates: nv
Data variables:
    THETA           (time, Z, latitude, longitude) float32 492GB dask.array<chunksize=(1, 25, 180, 360), meta=np.ndarray>
    SALT            (time, Z, latitude, longitude) float32 492GB dask.array<chunksize=(1, 25, 180, 360), meta=np.ndarray>
Attributes: (12/53)
    acknowledgement:                 This research was carried out by the Jet...
    author:                          Ian Fenty and Ou Wang
    cdm_data_type:                   Grid
    comment:                         Fields provided on a regular lat-lon gri...
    Conventions:                     CF-1.8, ACDD-1.3
    coordinates_comment:             Note: the global 'coordinates' attribute...
    ...                              ...
    source:                          The ECCO V4r4 state estimate was produce...
    standard_name_vocabulary:        NetCDF Climate and Forecast (CF) Metadat...
    summary:                         This dataset provides daily-averaged oce...
    time_coverage_duration:          P1D
    time_coverage_resolution:        P1D
    title:                           ECCO Ocean Temperature and Salinity - Da...

In [21]:
lat_range = (35, 45)
lon_range = (-135, -125)
time_range = (np.datetime64("2010-01-01"), np.datetime64("2011-01-01"))
data_subset = data.sel(latitude=slice(*lat_range), longitude=slice(*lon_range), time=slice(*time_range))
data_subset

<xarray.Dataset> Size: 58MB
Dimensions:         (time: 365, Z: 50, latitude: 20, longitude: 20, nv: 2)
Coordinates:
    time_bnds       (time, nv) datetime64[ns] 6kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    latitude_bnds   (latitude, nv) float32 160B dask.array<chunksize=(20, 2), meta=np.ndarray>
    longitude_bnds  (longitude, nv) float32 160B dask.array<chunksize=(20, 2), meta=np.ndarray>
    Z_bnds          (Z, nv) float32 400B dask.array<chunksize=(50, 2), meta=np.ndarray>
  * time            (time) datetime64[ns] 3kB 2010-01-01T12:00:00 ... 2010-12...
  * Z               (Z) float32 200B -5.0 -15.0 -25.0 ... -5.461e+03 -5.906e+03
  * latitude        (latitude) float32 80B 35.25 35.75 36.25 ... 44.25 44.75
  * longitude       (longitude) float32 80B -134.8 -134.2 ... -125.8 -125.2
Dimensions without coordinates: nv
Data variables:
    THETA           (time, Z, latitude, longitude) float32 29MB dask.array<chunksize=(1, 25, 20, 20), meta=np.ndarray>
    SALT            (time, Z, latitude, longitude) float32 29MB dask.array<chunksize=(1, 25, 20, 20), meta=np.ndarray>
Attributes: (12/53)
    acknowledgement:                 This research was carried out by the Jet...
    author:                          Ian Fenty and Ou Wang
    cdm_data_type:                   Grid
    comment:                         Fields provided on a regular lat-lon gri...
    Conventions:                     CF-1.8, ACDD-1.3
    coordinates_comment:             Note: the global 'coordinates' attribute...
    ...                              ...
    source:                          The ECCO V4r4 state estimate was produce...
    standard_name_vocabulary:        NetCDF Climate and Forecast (CF) Metadat...
    summary:                         This dataset provides daily-averaged oce...
    time_coverage_duration:          P1D
    time_coverage_resolution:        P1D
    title:                           ECCO Ocean Temperature and Salinity - Da...

In [ ]:
%%time
theta_mean_map = data_subset["THETA"].isel(Z=0).mean(dim="time").compute()

In [ ]:
theta_mean_map.plot()